In [1]:
import datetime
import os
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, DenseFeatures
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
CSV_COLUMNS = [
    'Unnamed: 0.1'
    ,'Flow ID'
    ,'Source IP'
    ,'Source Port'
    ,'Destination IP'
    ,'Destination Port'
    ,'Protocol'
    ,'Timestamp'
    ,'Flow Duration'
    ,'Total Fwd Packets'
    ,'Total Backward Packets'
    ,'Total Length of Fwd Packets'
    ,'Total Length of Bwd Packets'
    ,'Fwd Packet Length Max'
    ,'Fwd Packet Length Min'
    ,'Fwd Packet Length Mean'
    ,'Fwd Packet Length Std'
    ,'Bwd Packet Length Max'
    ,'Bwd Packet Length Min'
    ,'Bwd Packet Length Mean'
    ,'Bwd Packet Length Std'
    ,'Flow Bytes/s'
    ,'Flow Packets/s'
    ,'Flow IAT Mean'
    ,'Flow IAT Std'
    ,'Flow IAT Max'
    ,'Flow IAT Min'
    ,'Fwd IAT Total'
    ,'Fwd IAT Mean'
    ,'Fwd IAT Std'
    ,'Fwd IAT Max'
    ,'Fwd IAT Min'
    ,'Bwd IAT Total'
    ,'Bwd IAT Mean'
    ,'Bwd IAT Std'
    ,'Bwd IAT Max'
    ,'Bwd IAT Min'
    ,'Fwd PSH Flags'
    ,'Bwd PSH Flags'
    ,'Fwd URG Flags'
    ,'Bwd URG Flags'
    ,'Fwd Header Length'
    ,'Bwd Header Length'
    ,'Fwd Packets/s'
    ,'Bwd Packets/s'
    ,'Min Packet Length'
    ,'Max Packet Length'
    ,'Packet Length Mean'
    ,'Packet Length Std'
    ,'Packet Length Variance'
    ,'FIN Flag Count'
    ,'SYN Flag Count'
    ,'RST Flag Count'
    ,'PSH Flag Count'
    ,'ACK Flag Count'
    ,'URG Flag Count'
    ,'CWE Flag Count'
    ,'ECE Flag Count'
    ,'Down/Up Ratio'
    ,'Average Packet Size'
    ,'Avg Fwd Segment Size'
    ,'Avg Bwd Segment Size'
    ,'Fwd Header Length.1'
    ,'Fwd Avg Bytes/Bulk'
    ,'Fwd Avg Packets/Bulk'
    ,'Fwd Avg Bulk Rate'
    ,'Bwd Avg Bytes/Bulk'
    ,'Bwd Avg Packets/Bulk'
    ,'Bwd Avg Bulk Rate'
    ,'Subflow Fwd Packets'
    ,'Subflow Fwd Bytes'
    ,'Subflow Bwd Packets'
    ,'Subflow Bwd Bytes'
    ,'Init_Win_bytes_forward'
    ,'Init_Win_bytes_backward'
    ,'act_data_pkt_fwd'
    ,'min_seg_size_forward'
    ,'Active Mean'
    ,'Active Std'
    ,'Active Max'
    ,'Active Min'
    ,'Idle Mean'
    ,'Idle Std'
    ,'Idle Max'
    ,'Idle Min'
    ,'SimillarHTTP'
    ,'Inbound'
]
LABEL_COLUMN = 'Label'
UNWANTED_COLS = [
    'Unnamed: 0.1'
    ,'Flow ID'
    ,'Source IP'
    ,'Source Port'
    ,'Destination IP'
    ,'Destination Port'
    ,'Timestamp'
    ,'Flow Duration'
    ,'SimillarHTTP'
]


In [ ]:
INPUT_COLS = [
    'Total Fwd Packets'
    ,'Total Backward Packets'
    ,'Total Length of Fwd Packets'
    ,'Total Length of Bwd Packets'
    ,'Fwd Packet Length Max'
    ,'Fwd Packet Length Min'
    ,'Fwd Packet Length Mean'
    ,'Fwd Packet Length Std'
    ,'Bwd Packet Length Max'
    ,'Bwd Packet Length Min'
    ,'Bwd Packet Length Mean'
    ,'Bwd Packet Length Std'
    ,'Flow Bytes/s'
    ,'Flow Packets/s'
    ,'Flow IAT Mean'
    ,'Flow IAT Std'
    ,'Flow IAT Max'
    ,'Flow IAT Min'
    ,'Fwd IAT Total'
    ,'Fwd IAT Mean'
    ,'Fwd IAT Std'
    ,'Fwd IAT Max'
    ,'Fwd IAT Min'
    ,'Bwd IAT Total'
    ,'Bwd IAT Mean'
    ,'Bwd IAT Std'
    ,'Bwd IAT Max'
    ,'Bwd IAT Min'
    ,'Fwd PSH Flags'
    ,'Bwd PSH Flags'
    ,'Fwd URG Flags'
    ,'Bwd URG Flags'
    ,'Fwd Header Length'
    ,'Bwd Header Length'
    ,'Fwd Packets/s'
    ,'Bwd Packets/s'
    ,'Min Packet Length'
    ,'Max Packet Length'
    ,'Packet Length Mean'
    ,'Packet Length Std'
    ,'Packet Length Variance'
    ,'FIN Flag Count'
    ,'SYN Flag Count'
    ,'RST Flag Count'
    ,'PSH Flag Count'
    ,'ACK Flag Count'
    ,'URG Flag Count'
    ,'CWE Flag Count'
    ,'ECE Flag Count'
    ,'Down/Up Ratio'
    ,'Average Packet Size'
    ,'Avg Fwd Segment Size'
    ,'Avg Bwd Segment Size'
    ,'Fwd Header Length.1'
    ,'Fwd Avg Bytes/Bulk'
    ,'Fwd Avg Packets/Bulk'
    ,'Fwd Avg Bulk Rate'
    ,'Bwd Avg Bytes/Bulk'
    ,'Bwd Avg Packets/Bulk'
    ,'Bwd Avg Bulk Rate'
    ,'Subflow Fwd Packets'
    ,'Subflow Fwd Bytes'
    ,'Subflow Bwd Packets'
    ,'Subflow Bwd Bytes'
    ,'Init_Win_bytes_forward'
    ,'Init_Win_bytes_backward'
    ,'act_data_pkt_fwd'
    ,'min_seg_size_forward'
    ,'Active Mean'
    ,'Active Std'
    ,'Active Max'
    ,'Active Min'
    ,'Idle Mean'
    ,'Idle Std'
    ,'Idle Max'
    ,'Idle Min'
    ,'Inbound'
]

# Create input layer of feature columns
feature_columns = {
    colname: tf.feature_column.numeric_column(colname)
    for colname in INPUT_COLS
}

In [ ]:
model = Sequential([
    DenseFeatures(feature_columns=feature_columns.values()),
    Dense(units=1024, activation="relu", name="h1"),
    Dense(units=512, activation="relu", name="h2"),
    Dense(units=256, activation="relu", name="h3"),
    Dense(units=128, activation="relu", name="h4"),
    Dense(units=64, activation="relu", name="h5"),
    Dense(units=32, activation="relu", name="h6"),
    Dense(units=8, activation="relu", name="h7"),
    Dense(units=2, activation="softmax", name="output")
])

In [ ]:
algorithm = tf.keras.optimizers.RMSprop(
    learning_rate=0.001, rho=0.9, momentum=0.9, epsilon=1e-07, centered=False, name='RMSprop')
model.compile(optimizer=algorithm, loss="binary_crossentropy", metrics=["binary_crossentropy"])
